# Import the libraries and data

In [34]:
import pandas as pd
import SaniPath as sp

hh = pd.read_csv('rsrc/HOUSEHOLD_EXAMPLE.csv')
cc = pd.read_csv('rsrc/COMMUNITY_EXAMPLE.csv')
sc = pd.read_csv('rsrc/SCHOOL_EXAMPLE.csv')
col = pd.read_csv('rsrc/SAMPLE_EXAMPLE.csv')
lab = pd.read_csv('rsrc/LAB_EXAMPLE.csv')

# Make sure all R libraries are installed

Conda should handle most of the dependencies, but this can confirm.

In [52]:
reload(sp)
sp.RSetup('./r-requirements.txt') # wherever this r-requirements.txt file sits

SaniPath.py:130: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(any(x), 'No data seem to exist!')
SaniPath.py:133: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(getattr(self, "{}_data".format(data_name)).empty, 'Not enough data! Have you added everything?')
SaniPath.py:137: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(analysis_type in acceptable_options,
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Installing package into ‘/home/aw/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: also installing the dependencies ‘Formula’, ‘latticeExtra’, ‘acepack’, ‘gridExtra’, ‘viridis’, ‘mvtnorm’, ‘TH.data’, ‘sandwich’, ‘checkmate’, ‘lambda.r’, ‘futile.options’, ‘Hmisc’, ‘cmp

<SaniPath.RSetup instance at 0x7fb10fd01368>

# Setup the analysis class

Setup the analysis class instance with several things we need to know.  Not all are necessary all the time, but conducting analysis and generating a report, all are required. Otherwise, default values will be filled in. 

In [53]:
reload(sp)
analysis = sp.Analysis(
                 r_dir='./',
                 plot_dir = './plots/',
                 analysis_type = 'household',
                 # used for analysis ----
                 pathway_codes = {'p' : 2,
                                    'dw' : 3,
                                    'o' : 4,
                                    'l' : 7},
                 pathway_labels = {'p' : 'Produce',
                                 'dw' : 'Municipal and Drain Water',
                                 'o' : 'Ocean Water',
                                 'l' : 'Public Latrine'},
                 neighborhood_mapping = {'Jamaica Plain' : 1,
                                         'Brigthon' : 2},
                 # report specific arguments ----
                 city_name = 'Atlanta, GA',
                 lab_name = 'Bill Nye, Inc',
                 start_date = '2017-01-01',
                 lab_MF = False,
                 language = "English",
                 freq_thresh= 50
                      )


SaniPath.py:130: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(any(x), 'No data seem to exist!')
SaniPath.py:133: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(getattr(self, "{}_data".format(data_name)).empty, 'Not enough data! Have you added everything?')
SaniPath.py:137: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(analysis_type in acceptable_options,


Add data to the analysis object using the add_data() method.  Acceptable options match the forms we collect.  household, school, community, sample, and lab.

In [54]:
analysis.add_data('household', hh)
analysis.add_data('school', sc)
analysis.add_data('community', cc)
analysis.add_data('sample', col)
analysis.add_data('lab', lab)



# Frequency calculations (Pie Charts)

Use `compute_frequencies()` to calculate.  This depends on what `self.analysis_type` is set to at `__init__()`.  To check, look at analysis.analysis_type

In [38]:
print(analysis.analysis_type)

household


In [55]:
pie_chart_data = analysis.compute_frequencies()

Convert to and from R and Python using R's JSON reader.  It's much easier than trying to convert other ways.  We will need to store the results for caching purposes

In [40]:
print(analysis.get_frequencies())

[1] "[\n {\n \"sample\": \"Ocean Water\",\n\"age\": \"Adults\",\n\"neighborhood\": \"Neighborhood 1\",\n\"data\": [ 3, 2, 4, 2, 4 ],\n\"plot_name\": \"Neighborhood 1, Ocean Water\\nAdults (N= 5)\",\n\"s\": \"o\",\n\"neighb\": 1,\n\"pop\": \"a\",\n\"analysis_type\": \"household\",\n\"fn\": \"./plots/pie_1_o_a_household.png\" \n},\n{\n \"sample\": \"Ocean Water\",\n\"age\": \"Children\",\n\"neighborhood\": \"Neighborhood 1\",\n\"data\": [ 2, 2, 4, 1, 3 ],\n\"plot_name\": \"Neighborhood 1, Ocean Water\\nChildren (N= 5)\",\n\"s\": \"o\",\n\"neighb\": 1,\n\"pop\": \"c\",\n\"analysis_type\": \"household\",\n\"fn\": \"./plots/pie_1_o_c_household.png\" \n},\n{\n \"sample\": \"Produce\",\n\"age\": \"Adults\",\n\"neighborhood\": \"Neighborhood 1\",\n\"data\": [ 2, 3, 5, 4, 3 ],\n\"plot_name\": \"Neighborhood 1, Produce\\nAdults (N= 5)\",\n\"s\": \"p\",\n\"neighb\": 1,\n\"pop\": \"a\",\n\"analysis_type\": \"household\",\n\"fn\": \"./plots/pie_1_p_a_household.png\" \n},\n{\n \"sample\": \"Produce\

Import it back into the analysis class using `set_frequencies()`

In [41]:
analysis.set_frequencies(analysis.get_frequencies())

Each analysis type has these methods attached to them. 

If you look at the output, each object has an attribute called 'fn' which is short for filename.  This is a reference to where the plot was generated and saved to. By default, they are saved to `plots` inside the R working directory, but this can be set to any directory by setting the `plot_dir` argument at instantiation. 

![example plot](plots/pie_1_l_a_household.png)

The files are stored with the format [plot_type] [neighborhood_code] [pathway_code] [population_code] [survey_type].  Note that the histograms do not have [population_code] or [survey_type] since those are not population specific and are always collected with the Sample and Lab forms. 

In [42]:
import os
os.listdir('plots')

['ppl-1_l_c_school.png',
 'pie_1_o_c_household.png',
 'pie_1_d_c_school.png',
 'ppl_1_d_a_school.png',
 'hist_1_l.png',
 'hist_1_bw.png',
 'ppl-1_s_c_school.png',
 'hist_1_p.png',
 'ppl_1_s_a_school.png',
 'ppl-1_d_c_school.png',
 'hist_1_s.png',
 'ppl-1_d_a_school.png',
 'pie_1_bw_a_school.png',
 'ppl_1_sf_c_school.png',
 'pie_1_p_c_household.png',
 'pie_1_d_a_school.png',
 'hist_1_pa.png',
 'ppl-1_bw_c_school.png',
 'ppl_1_l_c_school.png',
 'pie_1_p_a_household.png',
 'ppl-1_p_a_school.png',
 'ppl_1_d_c_school.png',
 'ppl_1_p_c_school.png',
 'pie_1_sf_c_school.png',
 'pie_1_bw_c_school.png',
 'ppl-1_bw_a_school.png',
 'pie_1_o_c_school.png',
 'pie_1_l_c_school.png',
 'ppl_1_bw_a_school.png',
 'ppl-1_sf_c_school.png',
 'pie_1_p_a_school.png',
 'hist_1_sf.png',
 'pie_1_l_c_household.png',
 'pie_1_s_c_school.png',
 'ppl_1_bw_c_school.png',
 'pie_1_sf_a_school.png',
 'pie_1_s_a_school.png',
 'ppl_1_p_a_school.png',
 'ppl-1_p_c_school.png',
 'pie_1_f_c_school.png',
 'pie_1_o_a_household.p

# Concentrations

In [56]:
analysis.compute_concentrations()


In [44]:
print(analysis.concentrations)

[[1]]
[[1]]$s
[1] "p"

[[1]]$neighb
[1] 1

[[1]]$sample
[1] "Produce"

[[1]]$neighborhood
[1] "Neighborhood 1"

[[1]]$data
[1]    5 7894    5   NA  123

[[1]]$plot_name
[1] "Neighborhood 1, Produce\n(N=5)"

[[1]]$fn
[1] "./plots/hist_1_p.png"


[[2]]
[[2]]$s
[1] "l"

[[2]]$neighb
[1] 1

[[2]]$sample
[1] "Public Latrine"

[[2]]$neighborhood
[1] "Neighborhood 1"

[[2]]$data
[1] 5 5 5 5 5 5

[[2]]$plot_name
[1] "Neighborhood 1, Public Latrine\n(N=6)"

[[2]]$fn
[1] "./plots/hist_1_l.png"


[[3]]
[[3]]$s
[1] "dw"

[[3]]$neighb
[1] 1

[[3]]$sample
[1] "Municipal and Drain Water"

[[3]]$neighborhood
[1] "Neighborhood 1"

[[3]]$data
[1]  0.50 24.20 12.10  8.75  2.00  1.00  0.50

[[3]]$plot_name
[1] "Neighborhood 1, Municipal and Drain Water\n(N=7)"

[[3]]$fn
[1] "./plots/hist_1_dw.png"





In [45]:
x = analysis.get_concentrations()
analysis.set_concentrations(x)

# Exposure

Exposure calculations combine the other two types of analysis and simulate contamination based on the data collected.  It's pretty cool, but can take a while if running on a single core machine.  This is why we want to cache results. If you don't want to run in parallel set `parallel = False`, otherwise R will use as many cores as the machine has. This is taking a long time in Jupyter, but when running via console, it runs reasonably fast on a 4 core machine. Importing a stored version for now. 

In [57]:
# analysis.compute_exposures()
analysis.exposures = analysis._from_json('rsrc/pplplot_example.json')


![people plot example](plots/ppl-1_d_a_school.png)

# Report

In [58]:
analysis.compute_report()

/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from params (pos = 3):

    city_name, community_data, freq_thresh, household_data, lab_data,
    lab_MF, lab_name, language, neighborhood_mapping, pathway_codes,
    pathway_labels, ps_freq, sample_data, school_data, start_date


  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from params (pos = 5):

    city_name, community_data, freq_thresh, household_data, lab_data,
    lab_MF, lab_name, language, neighborhood_mapping, pathway_codes,
    pathway_labels, ps_freq, sample_data, school_data, start_date


  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from params (pos = 7):

[1] "Produce, Public Latrine, Ocean Water, and Municipal and Drain Water"
   Neighborhood Communities Surveyed Households Surveyed Schools Surveyed
1             3                    1                   0                0
2      Brigthon                    1                   0                0
3 Jamaica Plain                    2                   5                1
4         Total                    4                   5                1
   Neighborhood 1 10 5 8 9 Municipal and Drain Water Produce Public Latrine
1 Jamaica Plain 6  3 2 6 9                         7       5              6
2         Total 6  3 2 6 9                         7       5              6
[1] "hi"
  |....                                                             |   6%
  ordinary text without R code

  |........                                                         |  12%
label: unnamed-chunk-1 (with options) 
List of 1
 $ echo: logi FALSE

  |...........                                                     

/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

processing file: ./report_sections/Executive_Summary.Rmd

  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

processing file: ./report_sections/Introduction.Rmd

  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

processing file: ./report_sections/Methodology.Rmd

  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

processing file: ./report_sections/Results_and_Discussion.Rmd

  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

processing file: ./report_sections/Implicatio


  ordinary text without R code

  |.............................................................    |  94%
label: unnamed-chunk-8 (with options) 
List of 1
 $ child: chr "report_sections/Conclusion.Rmd"

  |.................................................................| 100%
   inline R code fragments


  |.................................................................| 100%
  ordinary text without R code




/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

processing file: ./report_sections/Conclusion.Rmd

  warnings.warn(x, RRuntimeWarning)


/home/aw/anaconda3/envs/sanipath-dev/bin/pandoc +RTS -K512m -RTS report.utf8.md --to docx --from markdown+autolink_bare_uris+ascii_identifiers+tex_math_single_backslash --output /home/aw/development/EpiTech/CGSW/RAT-shiny/python/report.docx --highlight-style tango 


/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: output file: report.knit.md


  warnings.warn(x, RRuntimeWarning)
/home/aw/anaconda3/envs/sanipath-dev/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Output created: report.docx

  warnings.warn(x, RRuntimeWarning)


The document should be stored in the root directory of R by default.  You can change this location with `out_dir`. 

R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fb10fcaabd8 / R:0x6037ed8>
[ListV..., ListV..., ListV..., ..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fb10fcaabd8 / R:0x6037ed8>
[ListV..., ListV..., ListV..., ..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fb10fcaabd8 / R:0x6037ed8>
[ListV..., ListV..., ListV..., ..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fb10fcaabd8 / R:0x6037ed8>
[ListV..., ListV..., ListV..., ..., ListV..., ListV..., ListV...]
  ...
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fb10fcaabd8 / R:0x6037ed8>
[ListV..., ListV..., ListV..., ..., ListV..., ListV..., ListV...]
R object with classes: ('list',) mapped to:
<ListVector - Python:0x7fb10fcaabd8 / R:0x6037ed8>
[ListV..., ListV..., ListV..., ..., ListV..., ListV..., ListV...]
R object with classes: ('lis